# Procesamiento de datos con Pandas

* *60 min* | Última modificación: Junio 22, 2019.

Este tutorial esta basado en https://es.hortonworks.com/tutorial/how-to-process-data-with-apache-hive/

El objetivo de este tutorial es demostrar las características de Pandas para analizar, procesar y filtrar los datos existentes en archivos planos de forma similar al lenguaje SQL estándar.


## Preparación

Los datos se encuentran almacenados en la carpeta `files/drivers` del directorio actual.

In [1]:
## importa la librería
import pandas as pd
import numpy as np

pd.set_option("display.notebook_repr_html", False)

In [2]:
##
## Lista los archivos.
##
!ls -1 files/drivers/*

ls: cannot access 'files/drivers/*': No such file or directory


El contenido de un archivo puede ser visualizado parcialmente usando el comando `head`. Se usa para realizar una inspección rápida del contenido de los archivos.

In [3]:
##
## Se imprime el final del archivo drivers
##
!head files/drivers/drivers.csv

head: cannot open 'files/drivers/drivers.csv' for reading: No such file or directory


In [4]:
!head files/drivers/timesheet.csv

head: cannot open 'files/drivers/timesheet.csv' for reading: No such file or directory


## Creación de la tabla `drivers`

A continuación se crea la tabla `drivers`como un DataFrame de Pandas.

In [5]:
drivers = pd.read_csv(
    "https://raw.githubusercontent.com/jdvelasq/playground/master/datasets/drivers/drivers.csv",
    sep=",",  # separador de campos
    thousands=None,  #  separador de miles para números
    decimal=".",
)  #  separador de los decimales para números

Se obtiene los primeros 10 registros de la tabla para realizar una inspección rápida de los datos y verificar que los datos fueron cargados correctamente.

In [6]:
drivers.head(10)

   driverId               name        ssn                            location  \
0        10  George Vetticaden  621011971                  244-4532 Nulla Rd.   
1        11     Jamie Engesser  262112338                  366-4125 Ac Street   
2        12        Paul Coddin  198041975           Ap #622-957 Risus. Street   
3        13        Joe Niemiec  139907145                 2071 Hendrerit. Ave   
4        14         Adis Cesir  820812209                 Ap #810-1228 In St.   
5        15       Rohit Bakshi  239005227                   648-5681 Dui- Rd.   
6        16         Tom McCuch  363303105    P.O. Box 313- 962 Parturient Rd.   
7        17        Eric Mizell  123808238  P.O. Box 579- 2191 Gravida. Street   
8        18          Grant Liu  171010151         Ap #928-3159 Vestibulum Av.   
9        19         Ajay Singh  160005158             592-9430 Nonummy Avenue   

  certified wage-plan  
0         N     miles  
1         N     miles  
2         Y     hours  
3         Y 

## Creación de la tabla `timesheet`

Se procede a crear la tabla y cargar los datos para el archivo `time_sheet`.

In [7]:
timesheet = pd.read_csv(
    "https://raw.githubusercontent.com/jdvelasq/playground/master/datasets/drivers/timesheet.csv",
    sep=",",  # separador de campos
    thousands=None,  #  separador de miles para números
    decimal=".",
)  #  separador de los decimales para números

In [8]:
timesheet.head(10)

   driverId  week  hours-logged  miles-logged
0        10     1            70          3300
1        10     2            70          3300
2        10     3            60          2800
3        10     4            70          3100
4        10     5            70          3200
5        10     6            70          3300
6        10     7            70          3000
7        10     8            70          3300
8        10     9            70          3200
9        10    10            50          2500

## Cantidad de horas y millas de cada conductor por año

En la siguiente consulta se desea obtener para cada conductor la cantidad de horas y millas por año.

In [9]:
sum_timesheet = timesheet.groupby("driverId").sum()
sum_timesheet.head()

          week  hours-logged  miles-logged
driverId                                  
10        1378          3232        147150
11        1378          3642        179300
12        1378          2639        135962
13        1378          2727        134126
14        1378          2781        136624

In [10]:
sum_timesheet = sum_timesheet[["hours-logged", "miles-logged"]]
sum_timesheet.head()

          hours-logged  miles-logged
driverId                            
10                3232        147150
11                3642        179300
12                2639        135962
13                2727        134126
14                2781        136624

In [11]:
# sum_timesheet['driverId'] = sum_timesheet.index.tolist()
# sum_timesheet.head()

## Unión de las tablas

El paso final consiste en agregar el nombre del conductor de la tabla `drivers` con la cantidad de horas y millas por año.

In [12]:
summary = pd.merge(sum_timesheet, drivers[["driverId", "name"]], on="driverId")
summary

    driverId  hours-logged  miles-logged                 name
0         10          3232        147150    George Vetticaden
1         11          3642        179300       Jamie Engesser
2         12          2639        135962          Paul Coddin
3         13          2727        134126          Joe Niemiec
4         14          2781        136624           Adis Cesir
5         15          2734        138750         Rohit Bakshi
6         16          2746        137205           Tom McCuch
7         17          2701        135992          Eric Mizell
8         18          2654        137834            Grant Liu
9         19          2738        137968           Ajay Singh
10        20          2644        134564         Chris Harris
11        21          2751        138719         Jeff Markham
12        22          2733        137550        Nadeem Asghar
13        23          2750        137980            Adam Diaz
14        24          2647        134461          Don Hilborn
15      

## Almacenamiento de los resultados

Finalmente, se almacena la tabla final obtenida en el archivo `tmp/summary.csv` para que otras aplicaciones puedan usar estos resultados.

In [13]:
!mkdir -p tmp
summary.to_csv(
    'tmp/summary.csv',
    sep = ',',
    header = True,
    index = False)

In [14]:
!head tmp/summary.csv

driverId,hours-logged,miles-logged,name
10,3232,147150,George Vetticaden
11,3642,179300,Jamie Engesser
12,2639,135962,Paul Coddin
13,2727,134126,Joe Niemiec
14,2781,136624,Adis Cesir
15,2734,138750,Rohit Bakshi
16,2746,137205,Tom McCuch
17,2701,135992,Eric Mizell
18,2654,137834,Grant Liu
